In [154]:
import numpy as np
import matplotlib.pyplot as plt
import math
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
%matplotlib qt

# Part 1
Some constants

In [155]:
charge = -1.9e-10 # coulombs

initial_vx = 20000 # mm/s
initial_vy = 0 # mm/s
initial_vz = 0 # mm/s
initial_x = 0 # Position, MM
initial_y = 0 # Position, MM
initial_z = 107.95 # Position, MM
initial_t = 0 # Seconds

droplet_diameter = 84e-3 # MM
droplet_density = 1000 # kg/m^3
droplet_mass = ( (4/3) * (droplet_diameter/2) ** 3) * math.pi * droplet_density
print_resolution = 300 # dpi

total_distance_travel = 3 # MM
plate_voltage = 0 # Volts, for part 1
plate_seperation = 1 # MM
plate_start = 1.25 # Position MM
plate_length = 0.5 # MM
plate_end = plate_start + plate_length # Position, MM
paper_end = 3 # MM



## Calculations For Question 1
We are assuming no voltage is applied to the plates. <br>
We are looking for how long it takes for the droplet to reach the paper. <br>
Due to the droplet being so small, we can negate air resistance. Therefore, the velocity will remain constant for X, Y and Z, for this case. <br>
While we have declared a part for the Y and Z component, we will not use it in this case, as there will be zero force applied and zero position changed in the Y and Z direction (for question 1)<br>

In [156]:
droplet = ({'position_x': [initial_x], 'position_y': [initial_y], 'position_z': [initial_z],
                'velocity_x': [initial_vx], 'velocity_y': [initial_vy], 'velocity_z': [0],
                'time': [initial_t], 'combined_velocity': [initial_vx]})

electric_field = plate_voltage / plate_seperation # Should be zero in this case, due to no plate voltage

step = 0
time_step = 1e-6

while (droplet['position_x'][step] <= paper_end):
    droplet['time'].append( time_step * (step + 1) )

    if (droplet['position_x'][step] < plate_start or droplet['position_x'][step] > plate_end): # Checks if the droplet is within the plates
        force_z = 0
    else: # If it is between the plates
        force_z = electric_field * charge # This should be zero in this case, due to there being no electric field between the plates

    droplet['velocity_x'].append(initial_vx) # Should not vary here
    droplet['velocity_z'].append( ( (force_z * time_step) / droplet_mass) + droplet['position_y'][step] )
    droplet['velocity_y'].append(initial_vy)

    droplet['position_z'].append( (droplet['velocity_z'][step] * time_step) + droplet['position_z'][step] )
    droplet['position_x'].append( (initial_vx * time_step) + droplet['position_x'][step] )
    droplet['position_y'].append(0) # It is known it will not change in this scenario
    combined_velocity = math.sqrt( (droplet['velocity_x'][step] ** 2) + (droplet['velocity_y'][step] ** 2) + (droplet['velocity_z'][step] ** 2))

    droplet['combined_velocity'].append(combined_velocity) 
    
    step = step + 1


## Plotting Question 1

In [157]:
fig1 = plt.figure(figsize=(8, 6))
ax1 = fig1.add_subplot(111, projection='3d')

x = np.linspace(plate_start, plate_end, 10)
y = np.linspace(-0.2, 0.2, 10)
X, Y = np.meshgrid(x, y)
# Creates first plate
Z = np.full_like(X, initial_z + 0.5)  
ax1.plot_surface(X, Y, Z, color='cyan', alpha=0.2)

# Creates second plate
Z = np.full_like(X, initial_z - 0.5)  
ax1.plot_surface(X, Y, Z, color='cyan', alpha=0.2)

# Similiar to before, but for the paper
y = np.linspace(-1, 1, 10)
z = np.linspace(initial_z - 1, initial_z + 1, 10)
Y, Z = np.meshgrid(y, z)
# Plot the paper
X = np.full_like(Y, paper_end) 
ax1.plot_surface(X, Y, Z, color='white', alpha=0.5)


ax1.set_title('Droplet Trajectory, when V=0')
ax1.set_xlabel('X Position (MM)')
ax1.set_zlabel('Z Position (MM)')
ax1.set_ylabel('Y Position (MM)')
ax1.set_ylim(np.min(droplet['position_y']) - 1, np.max(droplet['position_y']) + 1)
ax1.set_xlim(np.min(droplet['position_x']), np.max(droplet['position_x']) + 0.1)
ax1.set_zlim(np.min(droplet['position_z']) - 1, np.max(droplet['position_z']) + 1)
ax1.view_init(elev=5, azim=-105)


# fig2 = plt.figure(figsize=(8, 6))
# ax2 = fig2.add_subplot(111)
# ax2.set_title("Velocity over time")
# ax2.set_ylabel('Velocity (m/s)')
# ax2.set_xlabel('Time (s)')
# ax2.set_ylim(np.min(droplet['combined_velocity']) - 10 , np.max(droplet['combined_velocity']) + 10)
# ax2.set_xlim(np.min(droplet['time']), np.max(droplet['time']))

dot1 = ax1.scatter([], [], [], color='r', label='Position')
# dot2 = ax2.scatter([], [], color='b', label='Velocity')

def update_position(frame):
    dot1._offsets3d = (droplet['position_x'][frame:frame+1], droplet['position_y'][frame:frame+1], droplet['position_z'][frame:frame+1])
    return dot1,

# def update_velocity(frame):
#     dot2.set_offsets(np.c_[droplet['time'][frame:frame+1], droplet['combined_velocity'][frame:frame+1]])
#     return dot2,

ani1 = animation.FuncAnimation(fig1, update_position, frames=step, interval=10, blit=False, repeat=False)
# ani2 = animation.FuncAnimation(fig2, update_velocity, frames=step, interval=10, blit=False)

plt.show()

# Calculating time it took to to reach the paper
print(f"The time it took to reach the paper was {droplet['position_x'][-1]} seconds")


The time it took to reach the paper was 3.000000000000002 seconds


# Question 2
Simulate the act of drawing a vertical line (similar to the letter “I” ) along the center of the paper at 300 dpi resolution. How much time would it take to draw the letter ‘I’? <br>
In this case, the voltage needs to be dynamically varied accross each step. <br>
This will affect the Z position, which will be calcualted for each dot trajectory <br>

In [168]:

plate_voltage = 10000000 # This will by dynamically changing to draw an 'I'
droplets = [] # Keeps a array of droplets and their data
voltage_over_time = {'voltage': [plate_voltage], 'time': [0]} # This will keep track of the voltages that were applied to each droplet

dpmm = 12 # Roughly
num_droplets = dpmm * 1 # number of dots needed so that a 1mm long 'I' can be drawn
for i in range(num_droplets):
    droplet = {'position_x': [initial_x], 'position_y': [initial_y], 'position_z': [initial_z],
                'velocity_x': [initial_vx], 'velocity_y': [initial_vy], 'velocity_z': [0],
                'time': [initial_t], 'combined_velocity': [initial_vx]}
    droplets.append(droplet)

previous_vy = initial_vy
previous_vx = initial_vx
previous_px = initial_x
previous_py = initial_y

time_step = 1e-6
total_steps = 0
for i, droplet in enumerate(droplets):
    step = 0
    electric_field = plate_voltage / plate_seperation
    while (droplet['position_x'][step] <= paper_end):
        voltage_over_time['voltage'].append(plate_voltage)
        voltage_over_time['time'].append(time_step * (step + 1))
        droplet['time'].append( time_step * (total_steps + 1) )

        if (droplet['position_x'][step] < plate_start or droplet['position_x'][step] > plate_end): # Checks if the droplet is within the plates
            force_Z = 0
        else: # If it is between the plates
            force_Z = electric_field * charge 
        droplet['velocity_x'].append(initial_vx) # Should not vary here
        droplet['velocity_z'].append( ( (force_Z * time_step) / droplet_mass) + droplet['position_y'][step] )
        droplet['velocity_y'].append(initial_vy)

        droplet['position_z'].append( (droplet['velocity_z'][step] * time_step) + droplet['position_z'][step] )
        droplet['position_x'].append( (initial_vx * time_step) + droplet['position_x'][step] )
        droplet['position_y'].append(0) # It is known it will not change in this scenario
        combined_velocity = math.sqrt( (droplet['velocity_x'][step] ** 2) + (droplet['velocity_y'][step] ** 2) + (droplet['velocity_z'][step] ** 2))

        droplet['combined_velocity'].append(combined_velocity) 
        
        step = step + 1
        total_steps = total_steps + 1
    plate_voltage = plate_voltage - 100

# Plotting Question 2

In [169]:
fig1 = plt.figure(figsize=(8, 6))
ax1 = fig1.add_subplot(111, projection='3d')

x = np.linspace(plate_start, plate_end, 10)
y = np.linspace(-0.2, 0.2, 10)
X, Y = np.meshgrid(x, y)
# Creates first plate
Z = np.full_like(X, initial_z + 0.5)  
ax1.plot_surface(X, Y, Z, color='cyan', alpha=0.2)

# Creates second plate
Z = np.full_like(X, initial_z - 0.5)  
ax1.plot_surface(X, Y, Z, color='cyan', alpha=0.2)

# Similiar to before, but for the paper
y = np.linspace(-1, 1, 10)
z = np.linspace(initial_z - 1, initial_z + 1, 10)
Y, Z = np.meshgrid(y, z)
# Plot the paper
X = np.full_like(Y, paper_end) 
ax1.plot_surface(X, Y, Z, color='white', alpha=0.5)


ax1.set_title('Droplet trajectory when there is no voltage accross the plates')
ax1.set_xlabel('X Position (MM)')
ax1.set_zlabel('Z Position (MM)')
ax1.set_ylabel('Y Position (MM)')
ax1.set_ylim(-1, )
ax1.set_xlim(0, 3.1)
ax1.set_zlim(initial_z -2, initial_z + 1)
ax1.view_init(elev=5, azim=-105)

dots = []
for i, droplet in enumerate(droplets):
    print(droplet)
    dot = ax1.scatter([], [], [], color='r', label=f'Droplet {i+1}' if i == 0 else "")
    dots.append(dot)

def update_position(frame):
    current_droplet = frame // step 
    current_frame = frame % step
    
    if current_droplet < len(droplets):
        droplet = droplets[current_droplet]
        dots[current_droplet]._offsets3d = (droplet['position_x'][current_frame:current_frame+1],  droplet['position_y'][current_frame:current_frame+1], droplet['position_z'][current_frame:current_frame+1])
    return dots
total_frames = len(droplets) * step
ani1 = animation.FuncAnimation(fig1, update_position, frames=total_frames, interval=1, blit=False, repeat=False)

plt.show()


{'position_x': [0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12000000000000001, 0.14, 0.16, 0.18, 0.19999999999999998, 0.21999999999999997, 0.23999999999999996, 0.25999999999999995, 0.27999999999999997, 0.3, 0.32, 0.34, 0.36000000000000004, 0.38000000000000006, 0.4000000000000001, 0.4200000000000001, 0.4400000000000001, 0.46000000000000013, 0.48000000000000015, 0.5000000000000001, 0.5200000000000001, 0.5400000000000001, 0.5600000000000002, 0.5800000000000002, 0.6000000000000002, 0.6200000000000002, 0.6400000000000002, 0.6600000000000003, 0.6800000000000003, 0.7000000000000003, 0.7200000000000003, 0.7400000000000003, 0.7600000000000003, 0.7800000000000004, 0.8000000000000004, 0.8200000000000004, 0.8400000000000004, 0.8600000000000004, 0.8800000000000004, 0.9000000000000005, 0.9200000000000005, 0.9400000000000005, 0.9600000000000005, 0.9800000000000005, 1.0000000000000004, 1.0200000000000005, 1.0400000000000005, 1.0600000000000005, 1.0800000000000005, 1.1000000000000005, 1.1200000000000006, 1.14000